In [3]:
!pip install opacus

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [7]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Load PIMA Indian Diabetes dataset (or use sklearn diabetes dataset)
# Replace this with PIMA if using CSV; here, we use sklearn diabetes for demo
from sklearn.datasets import load_diabetes
data = load_diabetes()
X = data.data
y = (data.target > 140).astype(int)  # Convert to binary classification

# Preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Simple feedforward model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X.shape[1], 16)
        self.fc2 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    return correct / total

# Training function
def train_model(model, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

# Baseline: Without Differential Privacy
baseline_model = SimpleNN().to(device)
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

train_model(baseline_model, train_loader, optimizer, criterion)
acc_baseline = evaluate(baseline_model, test_loader)
print(f"Accuracy without DP: {acc_baseline:.4f}")

# With Differential Privacy
dp_model = SimpleNN().to(device)
optimizer_dp = torch.optim.Adam(dp_model.parameters(), lr=1e-3)
privacy_engine = PrivacyEngine()

dp_model, optimizer_dp, train_loader = privacy_engine.make_private(
    module=dp_model,
    optimizer=optimizer_dp,
    data_loader=train_loader,
    noise_multiplier=1.0,  # adjust based on epsilon desired
    max_grad_norm=1.0,
)

train_model(dp_model, train_loader, optimizer_dp, criterion)
acc_dp = evaluate(dp_model, test_loader)
print(f"Accuracy with DP (noise_multiplier=1.0): {acc_dp:.4f}")


Accuracy without DP: 0.7528


C:\Users\prasa\anaconda3\Lib\site-packages\opacus\privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
C:\Users\prasa\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Accuracy with DP (noise_multiplier=1.0): 0.4944
